# Example Notebook

Jupyter notebooks are a great tool for data analysis and prototyping. Each engineer will have their own folder under `notebooks/{username}/` to store Jupyter notebooks.

This notebook shows some examples.

Every notebook will start with this code cell. It moves up two directories from where the notebook is, so that we can execute code from the root directory, `transithealth/`. This allows us to import code from across project as well as read/write files using paths relative to the root directory.

In [1]:
import os
os.chdir("../../")

## Importing Code and Accessing Files

Now we can import code from the backend API and run it!

In [2]:
from api.metrics.rideshare import RideshareMetrics
from api.utils.testing import create_test_db

We can also refer to scripts from the offline pipeline using their relative paths from the root.

In [3]:
con, cur = create_test_db(
    scripts=[
        "./pipeline/load/rideshare.sql"
    ],
    tables={
        "rideshare": [
            { "n_trips": 7 },
            { "n_trips": 14 },
            { "n_trips": 3 }
        ]
    }
)

metric = RideshareMetrics(con)
actual = metric.get_max_trips()

expected = { "max_trips": 14 }

assert actual == expected

print(f"Actual:   {actual}")
print(f"Expected: {expected}")

Actual:   {'max_trips': 14}
Expected: {'max_trips': 14}


## Writing SQL

There are two ways to write SQL in a Jupyter notebooks:

1. With the `sqlite3` module built into Python.
2. With the SQL extension for Jupyter notebooks.

Here is an example using Python. The default fetched response does not include the columns, so we have a method in `api.utils.database` to help. We can also use Pandas to load the result into a DataFrame.

In [4]:
import sqlite3

con = sqlite3.connect("./pipeline/database.db")
cur = con.cursor()

rows = cur.execute("""
SELECT *
FROM rideshare
LIMIT 5
""").fetchall()

rows

[('2018-11-01', 1, 1, 500, '2018-44'),
 ('2018-11-01', 50, 43, 19, '2018-44'),
 ('2018-11-01', 50, 44, 17, '2018-44'),
 ('2018-11-01', 50, 46, 4, '2018-44'),
 ('2018-11-01', 50, 47, 6, '2018-44')]

In [5]:
import pandas as pd
from api.utils.database import rows_to_dicts

pd.DataFrame(rows_to_dicts(cur, rows))

,dropoff_community_area,n_trips,pickup_community_area,week,ymd
0,1,500,1,2018-44,2018-11-01
1,43,19,50,2018-44,2018-11-01
2,44,17,50,2018-44,2018-11-01
3,46,4,50,2018-44,2018-11-01
4,47,6,50,2018-44,2018-11-01


Jupyter also supports notebook extensions. This extension allows us to declare a cell as a SQL cell with `%%sql` on the first line of the cell, write queries directly in the cell body, and then view the result as a table.

In [6]:
%%capture
%load_ext sql
%sql sqlite:///pipeline/database.db

In [7]:
%%sql
SELECT *
FROM rideshare
LIMIT 5

 * sqlite:///pipeline/database.db
Done.


ymd,pickup_community_area,dropoff_community_area,n_trips,week
2018-11-01,1,1,500,2018-44
2018-11-01,50,43,19,2018-44
2018-11-01,50,44,17,2018-44
2018-11-01,50,46,4,2018-44
2018-11-01,50,47,6,2018-44


## Querying a Socrata Data Portal

Socrata SQL (SoQL) is a special dialect of SQL that we can use to access datasets from the City of Chicago data portal, as well as other data portals hosted on Socrata.

Below is an example code cell, which uses the `request` module to send a query and get the response as well as Pandas to display the result.

It can take a while to get a response, because you are sending a request to a remote server that will run your SoQL query against the entire dataset. That is why we write scripts in our offline pipeline to aggregate and download data before applying transformations locally.

In [8]:
import pandas as pd
import requests

dataset_json_url = "https://data.cityofchicago.org/resource/m6dm-c72p.json"
query = """
SELECT
    pickup_community_area,
    trip_seconds / 60 as trip_minutes
LIMIT 5
"""
r = requests.get(dataset_json_url, params={"$query": query})
pd.DataFrame(r.json())

,pickup_community_area,trip_minutes
0,68,23.5666666666666667
1,33,25.1000000000000000
2,25,30.4166666666666667
3,63,19.6333333333333333
4,1,7.3166666666666667
